<font size="6">**Import Libraries**</font>


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_hub as hub 

<font size="6">Image Data Preprocessing </font>


In [ ]:
normal_path = "/kaggle/input/brain-stroke-ct-image-dataset/Brain_Data_Organised/Normal"
stroke_path = "/kaggle/input/brain-stroke-ct-image-dataset/Brain_Data_Organised/Stroke"

In [ ]:
normal_folder = os.listdir(normal_path)
stroke_folder = os.listdir(stroke_path)

In [ ]:
normal_path = "/kaggle/input/brain-stroke-ct-image-dataset/Brain_Data_Organised/Normal"
stroke_path = "/kaggle/input/brain-stroke-ct-image-dataset/Brain_Data_Organised/Stroke"

data = []

for img_file in normal_folder:
    image = Image.open("/kaggle/input/brain-stroke-ct-image-dataset/Brain_Data_Organised/Normal/" + img_file)
    image = image.resize((224,224))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
    
for img_file in stroke_folder:
    image = Image.open("/kaggle/input/brain-stroke-ct-image-dataset/Brain_Data_Organised/Stroke/" + img_file)
    image = image.resize((224,224))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)

In [ ]:
normal_label = [0]*len(normal_folder)
stroke_label = [1]*len(stroke_folder)

In [ ]:
Target_label = normal_label + stroke_label
len(Target_label)

In [ ]:
x = np.array(data)
y = np.array(Target_label)

<font size="6">Split Data to Training and Test Sets</font>


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.10,shuffle=True)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train_s = x_train/255
x_test_s = x_test/255

<font size="6">Sample Data Visualization</font>


In [ ]:
class_labels = ["Normal","Stroke"]
plt.figure(figsize=(16,24))
for i in range(8):
    plt.subplot(6,4,i+1)
    plt.imshow(x_train[i])
    plt.title(f"Actual Label:{class_labels[y_train[i]]}")
    plt.axis("off")

<font size="6">CNN Model Structure</font>


In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),strides=1,padding="valid",activation="relu", input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(64,kernel_size=(3,3),strides=1,padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(128,kernel_size=(3,3),strides=1,padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy",
             metrics=["accuracy"])

model.summary()

In [ ]:
history = model.fit(x_train_s,y_train,batch_size=32,
         epochs=10, validation_data=(x_test_s,y_test))

<font size="6">Model Evaluation on Train and Test sets</font>


In [ ]:
loss, acc = model.evaluate(x_test_s,y_test)
print("Loss on Test Set:",loss)
print("Accuracy on Test Set:",acc)

loss, acc = model.evaluate(x_train_s,y_train)
print("Loss on Train Set:",loss)
print("Accuracy on Train Set:",acc)

In [ ]:
y_pred_test = model.predict(x_test_s)
y_pred_test_label = [1 if i>=0.5 else 0 for i in y_pred_test]

In [ ]:
print("Actual Label:",y_test[:10])
print("Predicted Label:",y_pred_test_label[:10])

<font size="6">Metrics Evaluation On Test Set</font>


In [ ]:
print("-----Metrics Evaluation On Test Set -----")
print()
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred_test_label))
print()
print("Classification Report:\n",classification_report(y_test,y_pred_test_label))

<font size="6">Image Predictions on Test Set</font>



In [ ]:
class_labels = ["Normal","Stroke"]
plt.figure(figsize=(16,32))
for i in range(8):
    plt.subplot(8,4,i+1)
    plt.imshow(x_test[i])
    plt.title(f"Actual:{class_labels[y_test[i]]}\nPredicted:{class_labels[y_pred_test_label[i]]}")
    plt.axis("off")